# 1、关于对话模型中消息(message)的使用
标准的对话模型的调用过程:

invoke()的输入可以是多种类型，典型的类型有:
- 字符串类型
- 消息列表
invoke()的输出类型:BaseMessage的子类:AIMessage

In [2]:
import os

import dotenv

from langchain_openai import ChatOpenAI


# 调用非对话模型:
# llms = OpenAI(...)

dotenv.load_dotenv() # 加载当前目录下的.env文件

os.environ["ARK_API_KEY"] = os.getenv("ARK_API_KEY")
os.environ["MODEL"] = os.getenv("MODEL")
os.environ["BASE_URL"] = os.getenv("BASE_URL")


# 调用对话模型:
chat_model =  ChatOpenAI(
    model_name=os.environ['MODEL'],
    base_url=os.environ['BASE_URL'],
    api_key=os.environ['ARK_API_KEY'],
    max_tokens=100
)

response = chat_model.invoke("什么是Go?")

print(response.content)



Go 又称 Golang，是由 Google 开发的一种开源、编译型、静态类型的编程语言，于 2009 年正式对外发布。它结合了高效开发和高性能运行的特点，在云计算、网络编程、分布式系统等领域应用广泛。以下是详细介绍：

### 语言特性
- **简单易学**：Go 语言的语法简洁，只有 25 个关键字，风格类似于 C 语言，对于有 C、C++ 或 Java 编程基础的开发者来说很容易上手。
```go
package main

import "fmt"

func main() {
    fmt.Println("Hello, World!")
}
```
- **高效性能**：Go 语言拥有高效的编译器，编译速度极快。同时，它的运行时性能也非常出色，具有垃圾回收机制，能有效管理内存，减少内存泄漏的风险。
- **并发编程**：Go 语言原生支持并发，通过 goroutine 和 channel 这两个强大的特性，可以轻松实现高效的并发程序。goroutine 是一种轻量级的线程，创建和销毁的开销极小；channel 则用于 goroutine 之间的通信和同步。
```go
package main

import (
    "fmt"
    "time"
)

func say(s string) {
    for i := 0; i < 5; i++ {
        time.Sleep(100 * time.Millisecond)
        fmt.Println(s)
    }
}

func main() {
    go say("world")
    say("hello")
}
```
- **内置标准库**：Go 拥有丰富的标准库，涵盖了网络编程、文件操作、加密解密、JSON 处理等各个方面，这使得开发者在开发过程中无需依赖第三方库就能完成很多常见的任务。
```go
package main

import (
    "fmt"
    "net/http"
)

func handler(w http.ResponseWriter, r *http.Request) {
    fmt.Fprintf(w, "Hello, you've requested: %s\n", r.URL.Path)
}

func 

在 LangChain 框架中，消息（Message）是构建对话和记忆系统的核心数据结构，用于在语言模型（LLM）、工具（Tools）、代理（Agents）以及用户之间传递信息。

LangChain 主要提供了以下几种内置的消息类型，它们都位于 `langchain_core.messages` 模块下。

---

### 核心消息类型

#### 1. `HumanMessage`
**作用**：代表来自用户或人类的消息。
**关键属性**：
- `content`: 消息的内容。可以是字符串、字符串列表或包含复杂内容的字典列表（例如，包含文本和图片的多模态内容）。
- `name` (可选): 发送消息的用户的名称，用于区分多个用户。
- `additional_kwargs` (可选): 提供此消息的原始提供者（如 OpenAI）所特有的任何额外参数。

**示例**：
```python
from langchain_core.messages import HumanMessage

message = HumanMessage(
    content="你好，请帮我总结这篇文章。",
    name="User_A"
)
```

#### 2. `AIMessage`
**作用**：代表来自人工智能助手或语言模型的回复。
**关键属性**：
- `content`: 助手的回复内容。通常是字符串，但也支持多模态。
- `name` (可选): 发出回复的助手的名称，用于区分多个助手。
- `tool_calls` (可选): 模型决定要调用的工具列表（对应 `ToolCall` 对象）。这是让模型主动使用工具的关键。
- `additional_kwargs` (可选): 原始提供者的额外参数。

**示例**：
```python
from langchain_core.messages import AIMessage

message = AIMessage(
    content="当然，请把文章内容提供给我。",
    # tool_calls=[...] # 模型可能会在这里指定要调用哪个工具
)
```

#### 3. `SystemMessage`
**作用**：用于设置对话的背景、指令或上下文。通常由开发者设置，用于引导模型的行为。
**关键属性**：
- `content`: 系统指令内容。
- `additional_kwargs` (可选): 原始提供者的额外参数。

**示例**：
```python
from langchain_core.messages import SystemMessage

message = SystemMessage(
    content="你是一个乐于助人的、简洁的助手。你的所有回复不得超过50个字。"
)
```

#### 4. `ToolMessage`
**作用**：作为工具调用结果的返回消息。当模型通过 `AIMessage.tool_calls` 请求调用工具后，需要用 `ToolMessage` 来将工具的执行结果返回给模型。
**关键属性**：
- `content`: 工具执行后返回的结果（通常是字符串形式的 JSON）。
- `tool_call_id`: **至关重要**。必须与触发此次工具调用的 `AIMessage.tool_calls` 中某个 `ToolCall` 的 `id` 相匹配，这样模型才能知道哪个工具的调用返回了此结果。
- `name` (可选): 被调用的工具的名称。

**示例**：
```python
from langchain_core.messages import ToolMessage

# 假设之前一个 AIMessage 的 tool_calls 中有一个 id 为 'call_abc123' 的工具调用
message = ToolMessage(
    content='{"result": "42"}', # 工具执行的结果
    tool_call_id='call_abc123', # 与请求的 ID 对应
    name="calculator_tool" # 工具名
)
```

#### 5. `FunctionMessage` (已逐渐被 `ToolMessage` 取代)
**作用**：在旧版本中，用于返回函数调用的结果。其功能与 `ToolMessage` 类似，但接口不同。在新代码中，推荐使用更通用的 `ToolMessage`。
**关键属性**：
- `content`: 函数执行的结果。
- `name`: 被调用的函数的名称。

---

### 与工具调用相关的辅助类

为了支持 `AIMessage` 中的 `tool_calls`，LangChain 还提供了以下类：

#### `ToolCall`
**作用**：一个数据类，表示模型请求的一次具体工具调用。
**关键属性**：
- `name`: 要调用的工具的名称。
- `args`: 一个字典，包含调用此工具时所需的参数。
- `id`: 此次调用的唯一标识符。**必须**与后续返回的 `ToolMessage` 中的 `tool_call_id` 保持一致。

**示例**：
```python
from langchain_core.messages import AIMessage, ToolCall

ai_msg = AIMessage(
    content="",
    tool_calls=[
        ToolCall(
            name="get_weather",
            args={"location": "Beijing"},
            id="call_001"
        )
    ]
)
# 当工具执行完成后，需要返回：
# ToolMessage(content="Sunny", tool_call_id="call_001")
```

---

### 消息的通用操作

所有消息类型都继承自 `BaseMessage`，因此共享一些通用方法和特性：

1.  **内容类型**：`content` 字段非常灵活，可以是：
    - `str`: 纯文本。
    - `List[Union[str, Dict]]`: 支持多模态内容。例如，OpenAI 的 GPT-4V 模型可以接受包含文本和图片 URL 的列表。
    ```python
    HumanMessage(content=[
        {"type": "text", "text": "请描述这张图片"},
        {"type": "image_url", "image_url": {"url": "https://example.com/image.png"}},
    ])
    ```

2.  **消息序列 (Chat History)**：对话历史本质上就是一个由各种 `Message` 对象组成的列表。
    ```python
    from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

    chat_history = [
        SystemMessage(content="你是助手"),
        HumanMessage(content="你好"),
        AIMessage(content="你好！很高兴为你服务。")
    ]
    ```

3.  **消息转换**：可以方便地在 LangChain 消息、原始 LLM 提供者的消息格式（如 OpenAI 的消息列表）以及字符串之间进行转换。

---

### 总结与使用场景

| 消息类型 | 发送者 | 主要目的 | 关键属性 |
| :--- | :--- | :--- | :--- |
| `HumanMessage` | 用户/客户端 | 提供用户输入 | `content`, `name` |
| `AIMessage` | 语言模型 | 返回模型响应或请求调用工具 | `content`, `tool_calls`, `name` |
| `SystemMessage` | 开发者/系统 | 设定模型角色和对话规则 | `content` |
| `ToolMessage` | 工具执行环境 | 返回工具调用结果给模型 | `content`, `tool_call_id` |
| `FunctionMessage` | 工具执行环境 | (旧版)返回函数调用结果 | `content`, `name` |

**典型工作流（使用工具）**：
1.  用户输入被封装为 `HumanMessage`。
2.  模型接收包含历史消息的列表，并返回一个 `AIMessage`。该消息可能包含 `tool_calls`（`ToolCall` 列表）。
3.  系统根据 `AIMessage.tool_calls` 中的信息执行相应的工具。
4.  每个工具的执行结果都被封装为一个 `ToolMessage`，其中 `tool_call_id` 与对应的 `ToolCall.id` 匹配。
5.  这些 `ToolMessage` 被追加到对话历史中，再次发送给模型。
6.  模型根据工具返回的结果生成最终的文本回复（另一个 `AIMessage`）。

掌握这些内置消息类型是使用 LangChain 构建复杂 AI 应用（尤其是带有工具调用功能的 Agent）的基础。



In [6]:
from langchain_core.messages import SystemMessage, HumanMessage
system_message = SystemMessage(content="你是一个英语教学的专家")
human_message = HumanMessage(content="帮我制定一个英语六级学习的计划")

messages = [system_message, human_message]
print(messages)

response = chat_model.invoke(messages)
print(type(response))
print(response.content)


[SystemMessage(content='你是一个英语教学的专家', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我制定一个英语六级学习的计划', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
以下是一份为期三个月的英语六级学习计划，你可以根据自己的实际情况进行调整。

### 第一阶段：基础积累（第 1 - 4 周）

#### 词汇积累
- **目标**：掌握六级核心词汇，扩充词汇量。
- **安排**：
    - 每天早上起床后和晚上睡觉前各安排 30 分钟背诵新单词，使用六级词汇书或背单词 APP，如百词斩、墨墨背单词等，每天背诵 50 - 60 个新单词。
    - 白天利用碎片化时间，如坐公交、排队时，复习当天新学的单词。
    - 每周安排一天对本周学习的单词进行集中复习和巩固，通过默写、填空等方式检验学习效果。

#### 语法巩固
- **目标**：系统复习英语语法知识，解决长难句理解问题。
- **安排**：
    - 每天花 30 - 45 分钟学习语法知识，可选择一本适合六级的语法书，如张满胜老师的《英语语法新思维》，按照章节顺序学习。
    - 结合语法知识，分析一些六级真题中的长难句，每周分析 5 - 7 个句子。

#### 听力入门
- **目标**：熟悉六级听力题型和语速，提高听力理解能力。
- **安排**：
    - 每天听 30 分钟英语听力材料，可以选择六级真题听力、英语广播（如 BBC、CNN）或英语有声读物。
    - 听的过程中，可以尝试做一些简单的听力练习，如填空、选择等，听完后对照原文，分析自己没听懂的地方。

### 第二阶段：专项提升（第 5 - 8 周）

#### 听力提升
- **目标**：提高听力答题准确率，掌握听力技巧。
- **安排**：
    - 每天进行 40 - 60 分钟的听力专项训练，使用六级真题听力，按照考试要求完成听力练习。
    - 做完听力练习后，认真分析错题，总结听力技巧，如关键词定位、同义替换等。
    - 每周安排 1 - 

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage
system_message = SystemMessage(content="你是一个英语教学的专家",additional_kwargs={"tool":"invoke_tool()"})
human_message = HumanMessage(content="帮我制定一个英语六级学习的计划")

messages = [system_message, human_message]
print(messages)

response = chat_model.invoke(messages)
print(type(response))
print(response.content)

[SystemMessage(content='你是一个英语教学的专家', additional_kwargs={'tool': 'invoke_tool()'}, response_metadata={}), HumanMessage(content='帮我制定一个英语六级学习的计划', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
以下是一份英语六级学习计划，你可以根据自己的实际情况进行调整。这个计划分为四个阶段，共12周，帮助你系统地准备六级考试。

### 第一阶段：基础积累（第1 - 3周）

#### 目标
扩充词汇量，熟悉六级常考语法，为后续学习打下坚实基础。

#### 每日学习安排
- **词汇学习**：
    - 每天背诵100个新单词，可以使用单词书（如《新东方六级词汇乱序版》）或手机APP（如百词斩、墨墨背单词）。早上背诵新单词，利用碎片化时间（如乘车、排队）复习。
    - 晚上花30 - 40分钟对当天背诵的单词进行默写检查，标记出没记住的单词，第二天重点复习。
- **语法复习**：
    - 每天花1 - 2小时复习六级常考语法知识，如时态、语态、虚拟语气、定语从句等。可以参考张满胜老师的《英语语法新思维》。
    - 做一些语法专项练习题，加深对语法规则的理解和运用。

#### 每周额外任务
- 阅读一篇六级难度的英文文章，如《纽约时报》《卫报》上的文章，标记出生词和长难句，进行分析和理解。

### 第二阶段：专项训练（第4 - 7周）

#### 目标
熟悉六级考试的各个题型，掌握解题技巧，提高答题能力。

#### 周一至周五每日学习安排
- **听力训练**：
    - 每天听一套六级听力真题，按照考试要求完成。听之前先快速浏览题目和选项，预测听力内容。
    - 听完后，对照听力原文，分析错题原因，总结听力技巧，如关键词定位、同义替换等。
    - 进行听力精听训练，逐句听写出听力原文，提高听力理解能力。
- **阅读训练**：
    - 每天做2 - 3篇不同类型的阅读理解真题（选词填空、长篇阅读、仔细阅读）。掌握不同题型的解题方法，如先看题目再读文章、根据关键词定位